In [217]:
import pandas as pd
from IPython.core.display_functions import display
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from scipy import stats
from scipy.stats import boxcox, yeojohnson
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

In [218]:
from main import read_xlsb

# download data
train_data_path = "./data/Training.xlsb"
train_sheet_name = 'Training'  # Укажите имя листа, который хотите прочитать
train_data = read_xlsb(train_data_path, train_sheet_name)

test_data_path = "./data/Test.xlsb"
test_sheet_name = 'Test'  # Укажите имя листа, который хотите прочитать
test_data = read_xlsb(test_data_path, test_sheet_name)

# создание train test dataframe
train_data_copy = train_data.copy()
X = train_data_copy.drop(["MARKER","ID"],axis = 1)
y = train_data_copy['MARKER']

#создание копии
X_copy = X.copy()

In [219]:
# удаление дублирующих строк
X_copy = X_copy.drop_duplicates()
print(X_copy)

              A         B       C       D         E         F     G     H  \
0      0.198778  0.099389    0.00  799.90  1.777556  0.888778  13.0  3.49   
1      0.043000  0.021264   49.97  173.03  0.384511  0.190143  13.0  3.49   
2      0.067073  0.067073    0.00  329.90  0.599818  0.599818  13.0  3.49   
3      0.052700  0.052700    0.00  235.65  0.471300  0.471300  13.0  3.49   
4      0.141880  0.141880    0.00  634.45  1.268900  1.268900  13.0  3.49   
...         ...       ...     ...     ...       ...       ...   ...   ...   
89729  0.027941  0.014844   25.00  474.00  0.557647  0.296250  20.0  0.01   
89730  0.120017  0.068581  179.96  720.03  1.200050  0.685743  10.0  0.00   
89731  0.153033  0.122427   29.01  550.99  0.459158  0.367327   3.0  0.00   
89732  0.107575  0.043030   53.82  215.18  0.537950  0.215180   5.0  0.00   
89733  0.095750  0.095750   57.48  172.42  0.287367  0.287367   3.0  0.00   

           I          J  ...                 O       P          Q    R  \
0

In [220]:

def encode_categorical_features(df):
    encoded_df = df.copy()

    # Шаг 1: Выбор категориальных признаков
    categorical_columns = df.select_dtypes(include=['object']).columns

    # Шаг 2: Закодировать категориальные признаки
    for column in categorical_columns:
        unique_values = df[column].nunique()
        if column == 'P':
            # Отдельный метод для столбца 'P'
            encoded_values = {
                '0 Zero': 0,
                '1 One': 1,
                '2 Two': 2,
                '3 Three': 3,
                'More than 3': 4
            }
            encoded_df[column] = df[column].map(encoded_values)
        elif unique_values <= 2:
            # One-hot encoding для признаков с <= 2 уникальными значениями
            encoder = LabelEncoder()
            encoded_df[column] = encoder.fit_transform(df[column])
        else:
            # Count encoding для признаков с > 2 уникальными значениями
            counter = df[column].value_counts().to_dict()
            encoded_df[column] = df[column].map(counter)

    return encoded_df



In [221]:

X_copy = encode_categorical_features(X)
print(X_copy)
X_copy.to_csv('encoded_X.csv')

              A         B       C       D         E         F     G     H  I  \
0      0.198778  0.099389    0.00  799.90  1.777556  0.888778  13.0  3.49  1   
1      0.043000  0.021264   49.97  173.03  0.384511  0.190143  13.0  3.49  1   
2      0.067073  0.067073    0.00  329.90  0.599818  0.599818  13.0  3.49  1   
3      0.052700  0.052700    0.00  235.65  0.471300  0.471300  13.0  3.49  1   
4      0.141880  0.141880    0.00  634.45  1.268900  1.268900  13.0  3.49  1   
...         ...       ...     ...     ...       ...       ...   ...   ... ..   
89729  0.027941  0.014844   25.00  474.00  0.557647  0.296250  20.0  0.01  0   
89730  0.120017  0.068581  179.96  720.03  1.200050  0.685743  10.0  0.00  1   
89731  0.153033  0.122427   29.01  550.99  0.459158  0.367327   3.0  0.00  0   
89732  0.107575  0.043030   53.82  215.18  0.537950  0.215180   5.0  0.00  1   
89733  0.095750  0.095750   57.48  172.42  0.287367  0.287367   3.0  0.00  0   

               J  ...      O  P      Q 

In [222]:
"""
def normalize_columns(df, small_value=1e-10):
    normalized_df = pd.DataFrame()

    for column in df.columns:
        data = df[column]

        # Применение всех перечисленных методов нормализации
        methods = [
            (np.log1p,),
            (lambda x: 1 / (x + small_value) if (x != 0).all() else x,),
            (np.sqrt,),
            (np.square,),
            (boxcox,),
            (yeojohnson,),
            (boxcox, yeojohnson),
            (np.log1p, boxcox),
            (np.log1p, yeojohnson),
            (np.log1p, boxcox, yeojohnson)
        ]

        # Переменные для выбора наилучшей комбинации методов
        best_methods = None
        best_p_value = 0.0

        for method_group in methods:
            transformed_data = data

            for method in method_group:
                if method in [boxcox, yeojohnson]:
                    if (transformed_data == 0).any():
                        transformed_data, _ = method(transformed_data + small_value)
                    else:
                        transformed_data, _ = method(transformed_data)
                else:
                    transformed_data = method(transformed_data)

            _, p_value = stats.normaltest(transformed_data)
            print("столбец:",column,"метод:",method,"текущий p_value:",p_value)
            if p_value > best_p_value:
                best_p_value = p_value
                best_methods = method_group

        # Применение наилучшей комбинации методов к столбцу
        transformed_data = data

        if best_methods:
            for method in best_methods:
                if method in [boxcox, yeojohnson]:
                    if (transformed_data == 0).any():
                        transformed_data, _ = method(transformed_data + small_value)
                    else:
                        transformed_data, _ = method(transformed_data)
                else:
                    transformed_data = method(transformed_data)

        normalized_df[column] = transformed_data

    return normalized_df

X_copy = normalize_columns(X_copy)
print(X_copy)
"""

'\ndef normalize_columns(df, small_value=1e-10):\n    normalized_df = pd.DataFrame()\n\n    for column in df.columns:\n        data = df[column]\n\n        # Применение всех перечисленных методов нормализации\n        methods = [\n            (np.log1p,),\n            (lambda x: 1 / (x + small_value) if (x != 0).all() else x,),\n            (np.sqrt,),\n            (np.square,),\n            (boxcox,),\n            (yeojohnson,),\n            (boxcox, yeojohnson),\n            (np.log1p, boxcox),\n            (np.log1p, yeojohnson),\n            (np.log1p, boxcox, yeojohnson)\n        ]\n\n        # Переменные для выбора наилучшей комбинации методов\n        best_methods = None\n        best_p_value = 0.0\n\n        for method_group in methods:\n            transformed_data = data\n\n            for method in method_group:\n                if method in [boxcox, yeojohnson]:\n                    if (transformed_data == 0).any():\n                        transformed_data, _ = method(tran

In [223]:
# обработка выбросов
def remove_outliers(df, num_columns_threshold=5):
    # Подсчитываем Q1 и Q3 для каждого столбца
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)

    # Рассчитываем межквартильный размах (IQR)
    iqr = q3 - q1

    # Определяем границы интервала для удаления выбросов
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Создаем маску для определения строк с выбросами в каждом столбце
    outliers_mask = ((df < lower_bound) | (df > upper_bound))

    # Подсчитываем количество столбцов с выбросами в каждой строке
    num_outliers = outliers_mask.sum(axis=1)

    # Создаем маску для определения строк с выбросами в 5 и более столбцах
    mask = (num_outliers >= num_columns_threshold)

    # Удаляем строки с выбросами
    df_cleaned = df[~mask]

    return df_cleaned

X_copy = remove_outliers(X_copy)
print(X_copy)


              A         B       C       D         E         F     G     H  I  \
0      0.198778  0.099389    0.00  799.90  1.777556  0.888778  13.0  3.49  1   
1      0.043000  0.021264   49.97  173.03  0.384511  0.190143  13.0  3.49  1   
2      0.067073  0.067073    0.00  329.90  0.599818  0.599818  13.0  3.49  1   
3      0.052700  0.052700    0.00  235.65  0.471300  0.471300  13.0  3.49  1   
4      0.141880  0.141880    0.00  634.45  1.268900  1.268900  13.0  3.49  1   
...         ...       ...     ...     ...       ...       ...   ...   ... ..   
89729  0.027941  0.014844   25.00  474.00  0.557647  0.296250  20.0  0.01  0   
89730  0.120017  0.068581  179.96  720.03  1.200050  0.685743  10.0  0.00  1   
89731  0.153033  0.122427   29.01  550.99  0.459158  0.367327   3.0  0.00  0   
89732  0.107575  0.043030   53.82  215.18  0.537950  0.215180   5.0  0.00  1   
89733  0.095750  0.095750   57.48  172.42  0.287367  0.287367   3.0  0.00  0   

               J  ...      O  P      Q 

In [224]:
# масштабирование
def robust_scale_dataframe(df):
    scaler = RobustScaler()
    scaled_data = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_data, columns=df.columns)
    return scaled_df
X_copy = robust_scale_dataframe(X_copy)
print(X_copy)

              A         B         C         D         E         F         G  \
0      0.555344  0.213817 -0.587765  0.816745  1.182216  0.664414  0.888889   
1     -0.497121 -0.567172  0.000118 -0.476624 -0.390651 -0.486815  0.888889   
2     -0.334481 -0.109236 -0.587765 -0.152967 -0.147551  0.188259  0.888889   
3     -0.431586 -0.252915 -0.587765 -0.347425 -0.292659 -0.023517  0.888889   
4      0.170932  0.638586 -0.587765  0.475386  0.607900  1.290789  0.888889   
...         ...       ...       ...       ...       ...       ...       ...   
87345 -0.598861 -0.631350 -0.293647  0.144343 -0.195166 -0.311969  1.666667   
87346  0.023219 -0.094159  1.529412  0.651956  0.530163  0.329848  0.555556   
87347  0.246286  0.444118 -0.246471  0.303190 -0.306368 -0.194847 -0.222222   
87348 -0.060840 -0.349583  0.045412 -0.389659 -0.217406 -0.445558  0.000000   
87349 -0.140732  0.177440  0.088471 -0.477882 -0.500336 -0.326607 -0.222222   

           H    I         J  ...         O    P    

In [225]:
# общий отбор признаков
# Функция для удаления признаков с малым разнообразием значений
def remove_low_variance_features(df, threshold=0.95):
    num_rows = len(df)
    low_variance_columns = [col for col in df.columns if (df[col].value_counts() / num_rows).max() > threshold]
    df_filtered = df.drop(columns=low_variance_columns)
    return df_filtered

# Функция для удаления признаков с высокой корреляцией
def remove_high_correlation_features(df, threshold=0.8):
    corr_matrix = df.corr()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    df_filtered = df.drop(columns=to_drop)
    return df_filtered

X_copy = remove_low_variance_features(X_copy)
print(X_copy)
X_copy = remove_high_correlation_features(X_copy)
print(X_copy)

              A         B         C         D         E         F         G  \
0      0.555344  0.213817 -0.587765  0.816745  1.182216  0.664414  0.888889   
1     -0.497121 -0.567172  0.000118 -0.476624 -0.390651 -0.486815  0.888889   
2     -0.334481 -0.109236 -0.587765 -0.152967 -0.147551  0.188259  0.888889   
3     -0.431586 -0.252915 -0.587765 -0.347425 -0.292659 -0.023517  0.888889   
4      0.170932  0.638586 -0.587765  0.475386  0.607900  1.290789  0.888889   
...         ...       ...       ...       ...       ...       ...       ...   
87345 -0.598861 -0.631350 -0.293647  0.144343 -0.195166 -0.311969  1.666667   
87346  0.023219 -0.094159  1.529412  0.651956  0.530163  0.329848  0.555556   
87347  0.246286  0.444118 -0.246471  0.303190 -0.306368 -0.194847 -0.222222   
87348 -0.060840 -0.349583  0.045412 -0.389659 -0.217406 -0.445558  0.000000   
87349 -0.140732  0.177440  0.088471 -0.477882 -0.500336 -0.326607 -0.222222   

           H    I         J  ...         O    P    